https://github.com/mahmoudparsian/pyspark-tutorial

## 1. Start

In [1]:
!export PYSPARK_PYTHON=/Users/shiwang/anaconda3/bin/python3

In [2]:
!pip install pyspark

google-cloud-bigquery 0.25.0 has requirement google-cloud-core<0.26dev,>=0.25.0, but you'll have google-cloud-core 0.28.1 which is incompatible.
apache-beam 2.2.0 has requirement protobuf<=3.3.0,>=3.2.0, but you'll have protobuf 3.5.2.post1 which is incompatible.
apache-beam 2.2.0 has requirement six<1.11,>=1.9, but you'll have six 1.11.0 which is incompatible.


In [3]:
import pyspark
from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkContext

In [4]:
sc = SparkContext()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [5]:
spark = SparkSession(sc)
spark

In [6]:
print(sc.version)

2.3.0


## 2. Basics

### RRD and its transformations

In [11]:
data = sc.parallelize(
    [('Amber', 22), ('Alfred', 23), ('Skye',4), ('Albert', 12), ('Amber', 9)]) #schema-less, allow diff types

In [12]:
data.take(2)

[('Amber', 22), ('Alfred', 23)]

In [14]:
#map
data.map(lambda row: (row[0], row[0])).take(2)

[('Amber', 'Amber'), ('Alfred', 'Alfred')]

In [15]:
#filter
data.filter(lambda row: row[1] < 20).take(3)

[('Skye', 4), ('Albert', 12), ('Amber', 9)]

In [24]:
#flatMap
data.flatMap(lambda row: (row[1], row[1]+1)).take(12)

[22, 23, 23, 24, 4, 5, 12, 13, 9, 10]

In [27]:
#distinct
data.map(lambda row: row[0]).distinct().collect()

['Amber', 'Skye', 'Alfred', 'Albert']

In [34]:
# Join
data_2 = sc.parallelize([('Amber', 2222), ('Alfred', 23)]) 
rdd = data.leftOuterJoin(data_2)
rdd.collect()

[('Amber', (22, 2222)),
 ('Amber', (9, 2222)),
 ('Skye', (4, None)),
 ('Alfred', (23, 23)),
 ('Albert', (12, None))]

In [36]:
rdd = data.join(data_2)
rdd.collect()

[('Amber', (22, 2222)), ('Amber', (9, 2222)), ('Alfred', (23, 23))]

In [46]:
#intersection
rdd = data.intersection(data_2)
rdd.collect()

[('Alfred', 23)]

### RRD and its actions

In [29]:
#count
data.count()

5

In [40]:
#sample
data.takeSample(num=3, withReplacement=False)

[('Alfred', 23), ('Amber', 9), ('Skye', 4)]

In [41]:
#reduce
data.map(lambda row: row[1]).reduce(lambda x, y: x + y)

70

In [42]:
data.map(lambda row: row[1]).reduce(lambda x, y: x / y)

0.0022141706924315623

In [47]:
#reduce by key
data.reduceByKey(lambda x, y: x + y).collect() # note the 1st one turple

[('Amber', 31), ('Skye', 4), ('Alfred', 23), ('Albert', 12)]

In [50]:
# count by key
data.countByKey().items()

dict_items([('Amber', 2), ('Alfred', 1), ('Skye', 1), ('Albert', 1)])

In [ ]:
# Save and Read
data.saveAsTextFile('./temp/data_key.txt') # partitioned into 8 pieces

In [58]:
def parseInput(row):
    import re
    
    pattern = re.compile(r'\(\'([A-Za-z]*)\', ([0-9]*)\)')
    row_split = pattern.split(row)
    
    return (row_split[1], int(row_split[2]))
    
data_key_reread = sc \
    .textFile('./temp/data_key.txt') \
    .map(parseInput)
    
data_key_reread.collect()

[('Amber', 22), ('Alfred', 23), ('Skye', 4), ('Albert', 12), ('Amber', 9)]

In [64]:
# for each
def function(x):
    print(x) #print to terminal
    
data.foreach(function)

### DataFrames

In [114]:
stringJSONRDD = sc.parallelize((""" 
  { "id": "123",
    "name": "Katie",
    "age": 19,
    "eyeColor": "brown"
  }""",
   """{
    "id": "234",
    "name": "Michael",
    "age": 22,
    "eyeColor": "green"
  }""", 
  """{
    "id": "345",
    "name": "Michael",
    "age": 23,
    "eyeColor": "green"
  }""")
)

In [115]:
# Create DataFrame from JSON
swimmersJSON = spark.read.json(stringJSONRDD)

In [116]:
#create table
swimmersJSON.createOrReplaceTempView("swimmersJSON")

In [117]:
#schema - auto-inferred
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



In [118]:
#show
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  Katie|
| 22|   green|234|Michael|
| 23|   green|345|Michael|
+---+--------+---+-------+



In [119]:
# SQL query
sql_query = "select * from swimmersJSON"
spark.sql(sql_query).collect()

[Row(age=19, eyeColor='brown', id='123', name='Katie'),
 Row(age=22, eyeColor='green', id='234', name='Michael'),
 Row(age=23, eyeColor='green', id='345', name='Michael')]

In [120]:
# DataFrame Query
swimmersJSON.select(swimmersJSON.age, swimmersJSON.eyeColor) \
            .filter(swimmersJSON.age > 20) \
            .show()

+---+--------+
|age|eyeColor|
+---+--------+
| 22|   green|
| 23|   green|
+---+--------+



In [121]:
# DataFrame Query
swimmersJSON.select(['age', 'eyeColor']) \
            .where(swimmersJSON.age > 20) \
            .show()

+---+--------+
|age|eyeColor|
+---+--------+
| 22|   green|
| 23|   green|
+---+--------+



In [122]:
import pyspark.sql.functions as fn

swimmersJSON.agg(
    fn.count('id').alias('count'),
    fn.countDistinct('id').alias('distinct')
).show()

+-----+--------+
|count|distinct|
+-----+--------+
|    3|       3|
+-----+--------+



In [123]:
# drop duplicated
df = swimmersJSON.dropDuplicates(
    subset=[c for c in swimmersJSON.columns \
            if c not in ['id','age']]) # exclude id column, drop duplicates
df.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 22|   green|234|Michael|
| 19|   brown|123|  Katie|
+---+--------+---+-------+



In [124]:
swimmersJSON.groupBy('eyeColor').count().show()

+--------+-----+
|eyeColor|count|
+--------+-----+
|   green|    2|
|   brown|    1|
+--------+-----+



In [125]:
swimmersJSON.describe('Age').show()

+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|                 3|
|   mean|21.333333333333332|
| stddev| 2.081665999466133|
|    min|                19|
|    max|                23|
+-------+------------------+



In [126]:
swimmersJSON.agg({'Age':'mean'}).show()

+------------------+
|          avg(Age)|
+------------------+
|21.333333333333332|
+------------------+



In [127]:
# Histrogram
hists = swimmersJSON.select('Age').rdd.flatMap(lambda row: row).histogram(2)

In [128]:
hists # size of bins calculated by workers before returning to driver

([19, 21, 23], [1, 2])

# 3. MLlib

# 4. TensorFrames

In [7]:
import tensorflow as tf
import tensorframes as tfs
from pyspark.sql import Row

ModuleNotFoundError: No module named 'tensorframes'